In [3]:
ls -alFH 

total 500
drwxr-xr-x. 10 llepotti zh   2048 Aug  2 15:38 ./
drwxr-xr-x.  6 llepotti zh   2048 Jul 17 21:03 ../
drwxr-xr-x.  4 llepotti zh   2048 Aug  2 11:12 ae/
drwxr-xr-x.  2 llepotti zh   2048 Aug  2 11:12 ae_custom_loss/
drwxr-xr-x.  3 llepotti zh   4096 Jul 18 16:56 dev/
drwxr-xr-x.  2 llepotti zh   2048 Aug  2 15:38 .ipynb_checkpoints/
drwxr-xr-x.  3 llepotti zh   2048 Aug  2 11:12 .old/
drwxr-xr-x.  2 llepotti zh   2048 Aug  2 11:12 sparse/
drwxr-xr-x.  2 llepotti zh   2048 Aug  2 11:12 test/
-rw-r--r--.  1 llepotti zh 489046 Aug  2 15:37 test.ipynb
drwxr-xr-x.  2 llepotti zh   2048 Aug  2 11:12 vae/
